### 절대경로와 상대경로

앞서 상대경로, 절대경로에 대한 설명을 잠깐 드렸는데요. 더 정확하게 이해를 하기 위해서 샌드위치 페이지에 있는 모든 url들을 가져와 볼게요.

In [ ]:
# 상대경로와 절대경로 - 그냥 검은색으로 되어 있는 부분은 상대경로, 링크로 되어 있는 것은 절대경로 
address_all = soup.find_all('div', 'sammy')
for item in address_all[:15]: 
    print(item.find('a')['href'])

/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-Roast-Beef/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Publican-Quality-Meats-PB-L/
https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Hendrickx-Belgian-Bread-Crafter-Belgian-Chicken-Curry-Salad/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Acadia-Lobster-Roll/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Birchwood-Kitchen-Smoked-Salmon-Salad/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Cemitas-Puebla-Atomica-Cemitas/
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Nana-Grilled-Laughing-Bird-Shrimp-and-Fried-Oyster-Po-Boy/
https://www.chicagomag.com/Chicago-Magazine/November-201

15줄만 뽑아 보았는데, 여기서 상대경로로 된 부분들이 많죠? 이것 이용해 바로 사이트를 열려면, 앞에 https://www.chicagomag.com 을 붙여주어야 합니다. 이렇게 묶어주기 위해서 urllib에 있는 urljoin을 사용할 거에요. 이 함수는 절대 경로라면 냅두고, 상대경로에만 앞의 주소를 붙여서 절대 경로로 전환시켜줘요.

In [ ]:
from urllib.parse import urljoin

urljoin('https://www.chicagomag.com', item.find('a')['href'])

'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Longman-and-Eagle-Wild-Boar-Sloppy-Joe/'

그러면 이제 모든 정보들을 가져와 보겠습니다.



In [ ]:
rank = []
main_menu = []
cafe_name = []
url_add =[]

list_soup = soup.find_all('div', 'sammy')

for item in list_soup:
    rank.append(item.find(class_='sammyRank').get_text())
    
    tmp_string = item.find(class_='sammyListing').get_text()
    
    main_menu.append(re.split(('\n|\r\n'), tmp_string)[0])
    cafe_name.append(re.split(('\n|\r\n'), tmp_string)[1])
    
    url_add.append(urljoin(url_base, item.find('a')['href']))

제대로 가져와졌는지 확인을 해보겠습니다.

In [ ]:
rank[:5]

['1', '2', '3', '4', '5']

In [ ]:
main_menu[:5]

['BLT', 'Fried Bologna', 'Woodland Mushroom', 'Roast Beef', 'PB&L']

In [ ]:
cafe_name[:5]

['Old Oak Tap', 'Au Cheval', 'Xoco', 'Al’s Deli', 'Publican Quality Meats']

In [ ]:
url_add[:5]

['https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/',
 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/',
 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/',
 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-Roast-Beef/',
 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Publican-Quality-Meats-PB-L/']

개수도 50개가 맞는지 확인해 볼게요!

In [ ]:
len(rank), len(main_menu), len(cafe_name), len(url_add)

(50, 50, 50, 50)

### DataFrame에 정보 담기


이렇게 웹 크롤링을 통해 구한 정보들을 모두 데이터 프레임에 담아 둡시다. 이제는 pandas의 DataFrame이 친숙하죠?ㅎㅎ

In [ ]:
import pandas as pd 

data = {'Rank': rank, 'Menu':main_menu, 'Cafe':cafe_name, 'URL':url_add}
df = pd.DataFrame(data)
df.head()

,Rank,Menu,Cafe,URL
0,1,BLT,Old Oak Tap,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Fried Bologna,Au Cheval,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Woodland Mushroom,Xoco,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Roast Beef,Al’s Deli,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,PB&L,Publican Quality Meats,https://www.chicagomag.com/Chicago-Magazine/No...


예히, 정보들이 잘 담겼네요. 우리가 원하는 칼럼 순으로 정보를 저장해주려면 아래와 같이 하면 됩니다.

In [ ]:
df = df[['Rank','Cafe','Menu','URL']]
df.head()

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...


잘 되었으니, 이를 우리 csv에다 저장해 둡니다. 이렇게 중간 저장을 잘 해놓지 않으면, 정보가 날라가 버리는 것은 당연하고, 웹 크롤링하는데 시간이 너무 많이 걸리면 다시 하기가 또 곤란해져요. 그러니 중간 저장을 습관화 해두는 것이 좋습니다.

In [ ]:
df.to_csv('03. best_sandwiches_list_chicago.csv', sep=',', encoding='utf-8')

이렇게 무사히 저장까지 해보았습니다.